# Bert Fine Tune Model

In [1]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/bert/tensorflow2/bert-en-uncased-l-10-h-512-a-8/2/saved_model.pb
/kaggle/input/bert/tensorflow2/bert-en-uncased-l-10-h-512-a-8/2/keras_metadata.pb
/kaggle/input/bert/tensorflow2/bert-en-uncased-l-10-h-512-a-8/2/assets/vocab.txt
/kaggle/input/bert/tensorflow2/bert-en-uncased-l-10-h-512-a-8/2/variables/variables.index
/kaggle/input/bert/tensorflow2/bert-en-uncased-l-10-h-512-a-8/2/variables/variables.data-00000-of-00001
/kaggle/input/bertfinetunev2/bert_v2/config.json
/kaggle/input/bertfinetunev2/bert_v2/tokenizer_config.json
/kaggle/input/bertfinetunev2/bert_v2/model.safetensors
/kaggle/input/bertfinetunev2/bert_v2/special_tokens_map.json
/kaggle/input/bertfinetunev2/bert_v2/vocab.txt
/kaggle/input/daigt-proper-train-dataset/train_drcat_03.csv
/kaggle/input/daigt-proper-train-dataset/train_drcat_02.csv
/kaggle/input/daigt-proper-train-dataset/train_drcat_04.csv
/kaggle/input/daigt-proper-train-dataset/train_drcat_01.csv
/kaggle/input/argugpt/argugpt.csv
/kaggle/input/argug

In [4]:
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import T5Tokenizer, T5ForConditionalGeneration
import pandas as pd
from sklearn.utils import resample

import torch
from transformers import DistilBertForSequenceClassification, AdamW
from tqdm import tqdm

from torch.utils.data import Dataset
from transformers import DistilBertTokenizer

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(f"Device Found : {device}")

Device Found : cuda


### Dataset 1

In [ ]:
# train = pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/train_essays.csv')
# test = pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/test_essays.csv')

# ## External Dataset 1
# ext_df1 = pd.read_csv('/kaggle/input/daigt-external-dataset/daigt_external_dataset.csv') [['id','source_text']]
# ext_df2 = pd.read_csv('/kaggle/input/argugpt/argugpt.csv')[['id','text']]

# ext_df1.rename(columns={'source_text':'text'}, inplace=True)
# ext_df1['generated'] = 1
# ext_df2['generated'] = 1


# all_train = pd.concat([ext_df2, ext_df1, train[['id','text','generated']]])
# all_train.head(3)

# df_majority = all_train[all_train['generated'] == 1]  # AI-written essays
# df_minority = all_train[all_train['generated'] == 0]  # Human-written essays

# # Upsample the minority class (human-written essays)
# df_minority_upsampled = resample(df_minority,
#                                  replace=True,      # sample with replacement
#                                  n_samples=6462,    # to match majority class
#                                  random_state=123)  # reproducible results

# # Combine majority class with upsampled minority class
# df_upsampled = pd.concat([df_majority, df_minority_upsampled])

# # Display new class counts
# print(df_upsampled['generated'].value_counts())

# train = df_upsampled.copy().reset_index(drop=True)
# train = train.rename(columns={'generated': 'label'})
# train.head()




# ## Dataset 2 Code 


# # Model Path
# model_path = '/kaggle/input/bertfinetunev2/bert_v2' 
# #model_path  = 'distilbert-base-uncased'

### Dataset 2

In [5]:
# External Dataset 2 
ext_df1 = pd.read_csv('/kaggle/input/daigt-proper-train-dataset/train_drcat_04.csv')
print(ext_df1['label'].value_counts())

train = ext_df1.copy().reset_index(drop=True)
train.head(3)

label
0    29792
1    14414
Name: count, dtype: int64


,essay_id,text,label,source,prompt,fold
0,E897534557AF,"In recent years, technology has had a profoun...",1,mistral7binstruct_v2,\nTask: Write an essay discussing the positive...,1
1,DFBA34FFE11D,Should students participate in an extracurricu...,0,persuade_corpus,NaN,2
2,af37ecf5,The electoral college is a symbol of mockery a...,0,train_essays,NaN,5


In [6]:
from sklearn.model_selection import train_test_split
# Split the DataFrame into train and test sets
#train_df, val_df = train_test_split(train, test_size=0.2, random_state=42)
train_df = train.copy()[['text','label']]
train_df.head(3)

,text,label
0,"In recent years, technology has had a profoun...",1
1,Should students participate in an extracurricu...,0
2,The electoral college is a symbol of mockery a...,0


### DataSet Class

In [7]:

class TextDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=512):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __getitem__(self, index):
        text = str(self.data.text.iloc[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        ids = inputs['input_ids']
        mask = inputs['attention_mask']

        return {
            'ids': ids.flatten(),
            'mask': mask.flatten(),
            'targets': torch.tensor(self.data.label.iloc[index], dtype=torch.long)
        }

    def __len__(self):
        return self.len


### DataLoader 

In [8]:
from torch.utils.data import DataLoader
#model_path = '/kaggle/input/bertfinetunev2/bert_v2' 

model_path = '/kaggle/working/fine_tune_bert_v3_10epoch'

tokenizer = DistilBertTokenizer.from_pretrained(model_path)
train_dataset = TextDataset(train_df, tokenizer, max_length=512) # max allowed length for bert
#val_dataset = TextDataset(val_df, tokenizer, max_length=512)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
#val_loader = DataLoader(val_dataset, batch_size=16)


model = DistilBertForSequenceClassification.from_pretrained(model_path, num_labels=2)
model.to(device)

optimizer = AdamW(model.parameters(), lr=1e-5)

print("Model Loaded Successfully.")


Model Loaded Successfully.


/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


### Fine Tune Bert

In [ ]:
for epoch in range(5):  # number of training epochs
    model.train()
    total_loss = 0
    progress_bar = tqdm(enumerate(train_loader), desc=f'Epoch {epoch}', total=len(train_loader), leave=False, disable=False)

    for batch_idx, batch in progress_bar:
        ids = batch['ids'].to(device)
        mask = batch['mask'].to(device)
        targets = batch['targets'].to(device)

        model.zero_grad()
        outputs = model(ids, attention_mask=mask, labels=targets)
        loss = outputs[0]
        total_loss += loss.item()
        loss.backward()
        optimizer.step()

        # Update progress bar
        progress_bar.set_postfix({'Training Loss': '{:.7f}'.format(total_loss / (batch_idx + 1))})

    progress_bar.close()
    print(f'Epoch: {epoch}, Training Loss: {total_loss / len(train_loader)}')
    
    # After Each Epoch Save Model 
    model.save_pretrained('/kaggle/working/fine_tune_bert_v3_10epoch')
    tokenizer.save_pretrained('/kaggle/working/fine_tune_bert_v3_10epoch')

    # Add code for validation here if needed


Epoch 0:  84%|████████▎ | 1157/1382 [25:25<04:53,  1.30s/it, Training Loss=0.0099664]

### Save Model

In [ ]:
model.save_pretrained('/kaggle/working/fine_tune_bert_v3_10epoch')
tokenizer.save_pretrained('/kaggle/working/fine_tune_bert_v3_10epoch')

### Prediction with Probabilities

In [ ]:
def predict_with_probability(model, tokenizer, texts, max_length=512):
    model.eval()  # Put the model in evaluation mode

    predictions = []
    probabilities = []
    with torch.no_grad():  # Disable gradient calculation for efficiency
        for text in texts:
            # Preprocess and tokenize the text
            inputs = tokenizer.encode_plus(
                text,
                add_special_tokens=True,
                max_length=max_length,
                padding='max_length',
                truncation=True,
                return_attention_mask=True,
                return_tensors='pt'
            )
            
            ids = inputs['input_ids'].to(device)
            mask = inputs['attention_mask'].to(device)

            # Get model predictions
            outputs = model(ids, attention_mask=mask)
            logits = outputs.logits

            # Convert logits to probabilities
            probs = torch.nn.functional.softmax(logits, dim=1)

            # Get the predicted class and its probability
            predicted_class = torch.argmax(probs, dim=1).cpu().numpy()
            max_prob = probs.max(dim=1).values.cpu().numpy()

            predictions.append(predicted_class[0])
            probabilities.append(max_prob[0])

    return predictions, probabilities


In [ ]:
sample_texts = ["Aaa bbb ccc.", "Bbb ccc ddd.", "CCC ddd eee."]
predictions, probs = predict_with_probability(model, tokenizer, sample_texts)

for text, pred, prob in zip(sample_texts, predictions, probs):
    print(f"Text: {text}, Predicted Class: {'AI' if pred == 1 else 'Human'}, Probability: {prob:.4f}")
